In [2]:
%%sh

curl -LO https://dumps.wikimedia.org/enwiki/latest/enwiki-latest-pages-articles.xml.bz2

Process is interrupted.


In [32]:
import bz2
import re, regex
import pdb
from tqdm import tqdm
import joblib

In [32]:
REGEX_PAGE_START = regex.compile('^\s+\<page\>')
REGEX_PAGE_END = regex.compile('^\s+\</page\>')
REGEX_TITLE = regex.compile('<title>(.+)</title>')
REGEX_PARENTHESE = re.compile('(（|\().*')

def clean_name(name):
    name = REGEX_PARENTHESE.sub('', name)
    return name.strip()

def save(names, path):
    with open(path, 'w') as f:
        rows = set(f'{name}\n' for name in names)
        f.writelines(rows)

def extract(input_path, output_path, REGEX_CATEGORY):
    entity_ls = []
    page_texts = []
    for line in tqdm(iter_wikidata(input_path)):
        if REGEX_PAGE_START.search(line):
            page_texts = []
            continue

        page_texts.append(line)
        if REGEX_PAGE_END.search(line):
            page = ''.join(page_texts)

            if not REGEX_CATEGORY.search(page):
                continue

            title = REGEX_TITLE.search(page).group(1)
            name = clean_name(title)

            if not is_valid_name(name, only_kanji_hiragana, exclude_short_name):
                continue
            entity_ls.append(name)

    save(entity_ls, output_path)

def iter_wikidata(input_path):
    if not input_path.endswith('bz2'):
        raise ValueError('extention of the input file must be bz2.')

    with bz2.BZ2File(input_path) as f:
        for line in f:
            yield line.decode('utf-8')

REGEX_CATEGORY = regex.compile('\[\[Category:Organizations')
extract('enwiki-latest-pages-articles.xml.bz2', 'output_organization.file')

REGEX_CATEGORY = regex.compile('\[\[Category:Living_people')
extract('enwiki-latest-pages-articles.xml.bz2', 'output_person.file')

In [32]:
def isCharAndSpace(string):
    return all(char.isalpha() or char.isspace() for char in string)

file_org = open('output_organization.file', 'r')
line_org = file_org.readlines()

file_per = open('output_person.file', 'r')
line_per = file_per.readlines()

In [33]:
print(line_org[0].strip()[-1])
name_org = list(set([name_.strip() for name_ in line_org if isCharAndSpace(name_.strip())]))
name_per = list(set([name_.strip() for name_ in line_per if isCharAndSpace(name_.strip())]))

l


In [34]:
print(len(name_org))
print(len(name_per))

24933
902007


In [37]:
joblib.dump(name_per, 'wiki_person.output')
joblib.dump(name_org, 'wiki_organization.output')

['wiki_organization.output']